In [1]:
import struct
import numpy as np
import tensorflow as tf
from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.layers import Conv2D, Add, ZeroPadding2D, UpSampling2D, Concatenate, MaxPooling2D
from keras.layers.merge import add, concatenate
from keras.models import Model

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import numpy as np
import keras as k
import keras.backend as K
from keras.layers import Input, Lambda
from keras.optimizers import Adam

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss, make_model,make_tiny_model
from yolo3.utils import get_random_data

from functools import wraps
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2

from yolo3.utils import compose
 

Using TensorFlow backend.


In [2]:
def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)

def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)


In [11]:
annotation_path = '3_CLASS_test.txt'
log_dir = 'logs/000/'
classes_path = '3_CLASS_test_classes.txt'
anchors_path = 'model_data/tiny_yolo_anchors.txt'
class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)
num_anchors = len(anchors)

input_shape = (416,416) # multiple of 32, hw

image_input = Input(shape=(None, None, 3))
# model = yolo_body(image_input,num_anchors//3,num_classes)
model_body = make_tiny_model(num_classes,num_classes)
# model_body.load_weights("final_weight_1106.h5")
h, w = input_shape

# y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
#     num_anchors//3, num_classes+5)) for l in range(3)]

y_true = [Input(shape=(h//{0:32, 1:16}[l], w//{0:32, 1:16}[l], \
    num_anchors//2, num_classes+5)) for l in range(2)]


model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})([*model_body.output, *y_true])
model = Model([model_body.input, *y_true], model_loss)

# model.summary()



In [12]:
print(model.output)

logging = TensorBoard(log_dir=log_dir)
checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

val_split = 0.1
with open(annotation_path) as f:
    lines = f.readlines()
    
    
np.random.shuffle(lines)
num_val = int(len(lines)*val_split)
num_train = len(lines) - num_val
print(num_classes)
print(num_val)
print(num_train)

Tensor("yolo_loss_4/add_11:0", shape=(), dtype=float32)
3
5202
46821


In [13]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_66 (Conv2D)              (None, None, None, 1 432         input_22[0][0]                   
__________________________________________________________________________________________________
batch_normalization_56 (BatchNo (None, None, None, 1 64          conv2d_66[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_56 (LeakyReLU)      (None, None, None, 1 0           batch_normalization_56[0][0]     
__________________________________________________________________________________________________
max_poolin

In [14]:
model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred})
    
batch_size = 8
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
        steps_per_epoch=max(1, num_train//batch_size),
        validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
        validation_steps=max(1, num_val//batch_size),
        epochs=35,
        callbacks=[logging, checkpoint])

Train on 46821 samples, val on 5202 samples, with batch size 8.
Epoch 1/35
5852/5852 [==============================] - 3258s 557ms/step - loss: 54.6710 - val_loss: 22.4187
Epoch 2/35
5852/5852 [==============================] - 3273s 559ms/step - loss: 19.5439 - val_loss: 18.6605
Epoch 3/35
5852/5852 [==============================] - 3186s 544ms/step - loss: 17.7765 - val_loss: 17.5305
Epoch 4/35
5852/5852 [==============================] - 3206s 548ms/step - loss: 16.9458 - val_loss: 16.7414
Epoch 5/35
5852/5852 [==============================] - 3214s 549ms/step - loss: 16.4148 - val_loss: 16.3985
Epoch 6/35
5852/5852 [==============================] - 3194s 546ms/step - loss: 16.0496 - val_loss: 16.6905
Epoch 7/35
5852/5852 [==============================] - 3201s 547ms/step - loss: 15.7049 - val_loss: 15.6234
Epoch 8/35
5852/5852 [==============================] - 3187s 545ms/step - loss: 15.5766 - val_loss: 15.6916
Epoch 9/35
5852/5852 [==============================] - 3191s 54

In [15]:
model.save_weights("highway_tiny_weight.h5")